![This is an image](Quant-Trading.jpg)

<font size="3">
Please visit our website <a href="https://www.quant-trading.co" target="_blank">quant-trading.co</a> for more tools on quantitative finance and data science.
</font>

# **PRINCIPAL COMPONENT ANALYSIS FOR INTEREST RATES**

## **Use PCA to calculate the Value at Risk of a Portfolio**

<font size="3"> In this notebook we are showing how to use Principal Component Analysis for Interest Rates. In particular we are going to show how PCA can be used to calculate the Value at Risk - VaR of a portfolio of interest rate sensitive assets, such as bonds or swaps. In a previous <a href="https://quant-trading.co/principal-component-analysis-for-interest-rates/" target="_blank">notebook</a> we explained what PCA is, how you can calculate it and provided a few examples on how it is applied on interest rate curves. The application of PCA to calculate VaR is easy to implement and can give you a quick answer to the level of risk of your interest rate portfolio.<br><br>

## **First thing to do: Import all the required libraries**

<font size="3"> Here we will be using libraries such as pandas or numpy for calculations. We will also use matplotlib to make charts. We will use pandas_datareader to download data from the FRED API. If you would like to know how to do that, please take a look at our <a href="https://quant-trading.co/how-to-download-data-from-federal-reserve-fred-api/" target="_blank">notebook</a> on downloading economic and financial Data from FRED, the FED of St Louis. We also need libraries such as sklearn or scipy.optimize for more advanced calculations<br><br>  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as font_manager
from matplotlib.ticker import FormatStrFormatter
import matplotlib.dates as mdates
formatter = mdates.DateFormatter("%Y-%m-%d")
import sklearn
from sklearn import decomposition
from sklearn import datasets
import datetime
import pandas_datareader.data as web
from math import pi, cos, sin
from scipy.optimize import minimize

## **Download yield from treasury bonds**

<font size="3">We need to download data for treasury bonds with different maturities. Fortunately, we can find series of constant maturity treasuries on FRED. That would be a theoretical yield, since all the treasuries mature at some point in time, but the Federal Reserve uses a methodology to track the yields of constant maturity bonds. We will get that information from 1 year up to 30 years to maturity using the following script:<br><br>  

In [2]:
#Parámetros de fechas
start = datetime.datetime(1980, 1, 1)
end = datetime.datetime(2024, 10, 31)

#US Treasuries

US_Rates = web.DataReader(['DGS1','DGS2','DGS3','DGS5','DGS7','DGS10','DGS30'], 'fred', start, end)
US_Rates.dropna(inplace=True)
US_Rates.reset_index(inplace=True)
US_Rates.head()

,DATE,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS30
0,1980-01-02,11.89,11.39,10.77,10.52,10.51,10.50,10.23
1,1980-01-03,12.25,11.45,10.79,10.54,10.59,10.60,10.31
2,1980-01-04,12.25,11.47,10.81,10.62,10.63,10.66,10.34
3,1980-01-07,12.07,11.35,10.72,10.56,10.60,10.63,10.35
4,1980-01-08,11.94,11.26,10.67,10.52,10.55,10.57,10.28


## **Calculate daily moves using the differences**

<font size="3">Now we can calculate the daily movements of interest simply by substracting yesterday's rate from today's rate. You can see how it is done below:<br><br>  

In [3]:
US_Rates['diff_DGS1'] = US_Rates['DGS1'] - US_Rates['DGS1'].shift(1)
US_Rates['diff_DGS2'] = US_Rates['DGS2'] - US_Rates['DGS2'].shift(1)
US_Rates['diff_DGS3'] = US_Rates['DGS3'] - US_Rates['DGS3'].shift(1)
US_Rates['diff_DGS5'] = US_Rates['DGS5'] - US_Rates['DGS5'].shift(1)
US_Rates['diff_DGS7'] = US_Rates['DGS7'] - US_Rates['DGS7'].shift(1)
US_Rates['diff_DGS10'] = US_Rates['DGS10'] - US_Rates['DGS10'].shift(1)
US_Rates['diff_DGS30'] = US_Rates['DGS30'] - US_Rates['DGS30'].shift(1)
US_Rates.dropna(inplace=True)
US_Rates

,DATE,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS30,diff_DGS1,diff_DGS2,diff_DGS3,diff_DGS5,diff_DGS7,diff_DGS10,diff_DGS30
1,1980-01-03,12.25,11.45,10.79,10.54,10.59,10.60,10.31,0.36,0.06,0.02,0.02,0.08,0.10,0.08
2,1980-01-04,12.25,11.47,10.81,10.62,10.63,10.66,10.34,0.00,0.02,0.02,0.08,0.04,0.06,0.03
3,1980-01-07,12.07,11.35,10.72,10.56,10.60,10.63,10.35,-0.18,-0.12,-0.09,-0.06,-0.03,-0.03,0.01
4,1980-01-08,11.94,11.26,10.67,10.52,10.55,10.57,10.28,-0.13,-0.09,-0.05,-0.04,-0.05,-0.06,-0.07
5,1980-01-09,11.87,11.24,10.68,10.53,10.56,10.58,10.29,-0.07,-0.02,0.01,0.01,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11205,2024-10-24,4.25,4.07,4.02,4.03,4.12,4.21,4.47,-0.02,0.00,-0.01,-0.02,-0.02,-0.03,-0.04
11206,2024-10-25,4.29,4.11,4.05,4.07,4.15,4.25,4.51,0.04,0.04,0.03,0.04,0.03,0.04,0.04
11207,2024-10-28,4.28,4.12,4.09,4.11,4.19,4.28,4.53,-0.01,0.01,0.04,0.04,0.04,0.03,0.02
11208,2024-10-29,4.29,4.11,4.08,4.11,4.18,4.28,4.52,0.01,-0.01,-0.01,0.00,-0.01,0.00,-0.01


## **Principal Component Analysis for interest rates**

<font size="3">We can use sklearn, with its a method decomposition, to get the A matrix which is quite important in PCA. Below we show an example on how to do that using sklearn.<br><br>  

In [4]:
pca = decomposition.PCA(n_components=7)
matrix_A = pca.fit(US_Rates[['diff_DGS1','diff_DGS2','diff_DGS3','diff_DGS5','diff_DGS7','diff_DGS10','diff_DGS30']]).components_
matrix_A

array([[-0.37935958, -0.400595  , -0.40384002, -0.39844359, -0.38710534,
        -0.36144955, -0.30540069],
       [-0.66071573, -0.34181553, -0.1247841 ,  0.1426303 ,  0.30164856,
         0.37058712,  0.42705472],
       [-0.5651073 ,  0.29065997,  0.42217094,  0.28278505,  0.0813303 ,
        -0.12472086, -0.56196523],
       [-0.31110197,  0.58874255,  0.1380183 , -0.37249274, -0.3564661 ,
        -0.11701119,  0.5079746 ],
       [ 0.05274804, -0.53153838,  0.70479395,  0.08141229, -0.2904425 ,
        -0.24831576,  0.25554757],
       [ 0.02499255, -0.09494626,  0.35188044, -0.71336417,  0.16601766,
         0.51300053, -0.25869071],
       [-0.00183368, -0.00607788,  0.05924916, -0.29438865,  0.71662835,
        -0.61477006,  0.13522622]])

## **Information contained in A matriz**

<font size="3">We can use the information from matrix_A to calculate the Value at Risk of a portfolio of interest rate sensitive assets, like bonds or swaps. For example, the first element in this matrix is -0.37935958. Since we are working with rates in the format 5.00 to indicate a 5% interest rate, we need to divide this number by 100 to understand what it means. In this case it means that the 1Y interest rate will move in approximately -0.38 basis points when the first principal component moves in one unit. Correspondingly the 30Y interest rate will move in -0.3054 basis point when the first principal component moves in one unit.<br><br> 
    
<font size="3">Let's suppose we have a portfolio with the following exposures to 1 basis point interest rate risk: <br><br> 
<font size="3">1Y USD 35.000<br><br> 
<font size="3">5Y USD 55.000<br><br> 
<font size="3">10Y USD 85.000<br><br> 
    
    
<font size="3">The exposure to the first principal component will be: 35.000*(-0.3793) + 55.000*(-0.3984) + 85.000*(-0.3614) <br><br> 

In [16]:
35000*(-0.3793) + 55000*(-0.3984) + 85000*(-0.3614) 

-65906.5

<font size="3">We get USD -65.906.5 which is the dollar interest rate exposure to the first principal component. we can do the same for the second principal component and we will get the following:<br><br> 
    
<font size="3">The exposure to the second principal component will be: <br><br>
    
    
<font size="3"> 35.000*(-0.6607) + 55.000*(0.1426) + 85.000*(-0.3705) <br><br> 

In [17]:
35000*(-0.6607) + 55000*(0.1426) + 85000*(-0.3705) 

-46774.0

## **Change in portfolio's value**

<font size="3">The total change in the portfolio's value due to the first two principal components of the interest rate curve will be: <br><br>                                                                                        
                                                                                                                                 <font size="3"> Change = -65.906*F1 - 46774*F2<br><br> 

## **VaR calculation**

<font size="3">We can get the variance of each principal component using the function get_covariance as following:<br><br>    

In [9]:
cov_matrix = pca.get_covariance()
cov_matrix 

array([[0.00703238, 0.00604412, 0.00563803, 0.00513901, 0.00476295,
        0.00435277, 0.00363018],
       [0.00604412, 0.00656998, 0.00613456, 0.00570582, 0.00534734,
        0.00488166, 0.00396015],
       [0.00563803, 0.00613456, 0.00638406, 0.00592557, 0.00560299,
        0.00513862, 0.00418137],
       [0.00513901, 0.00570582, 0.00592557, 0.00614031, 0.00582642,
        0.00539401, 0.004455  ],
       [0.00476295, 0.00534734, 0.00560299, 0.00582642, 0.00594703,
        0.0054761 , 0.00461423],
       [0.00435277, 0.00488166, 0.00513862, 0.00539401, 0.0054761 ,
        0.00536772, 0.0045512 ],
       [0.00363018, 0.00396015, 0.00418137, 0.004455  , 0.00461423,
        0.0045512 , 0.00446375]])

<font size="3">Here, we can observe that the first element of the covariance matrix is 0.00703238. If take the square root of that number we get:<br><br> 

In [29]:
0.00703238**0.5

0.08385928690371747

<font size="3">Since we are working with rates in the format 5.00 to indicate a 5% interest rate, we need to divide this number by 100 to understand what it means. If we do that we get 0.000838, which means 8.38 basis points. This means, that 1 standard deviation of the first principal component corresponds to 8.38 basis points. If we do the same for the second component we get 7.77 basis points<br><br> 

In [30]:
0.00604412**0.5

0.07774393867048414

<font size="3">Then, we can calculate the exposure of the portfolio to a move of 1 standard deviation in interest rates. Since principal component are orthogonal, we don't need to use correlation into account since it is zero. Therefore we get:<br><br> 

<font size="5"> 

$Change = \sqrt(65906^2 * 8.38^2 + 46774^2 * 7.77^2$)

<br>

In [31]:
(65906**2 * 8.38**2 + 46774**2 * 7.77**2)**0.5

661143.7214148213

<font size="3">The change in the portfolio due to 1 standard deviation daily move in interest rates is USD 661.143. Since we are interested in the Value at Risk Calculation we need to scale this number. People usually work with 3 standard deviations, so we need to multiply this number by 2.33 if we work under the assumption of a normal distribution. So we will get:<br><br> 

<font size="5"> 

$VaR = \sqrt(65906^2 * 8.38^2 + 46774^2 * 7.77^2$) * 2.33

<br>

In [32]:
(65906**2 * 8.38**2 + 46774**2 * 7.77**2)**0.5 * 2.33

1540464.8708965338

<font size="3">The Value at Risk of VaR of this portfolio equals USD 1.540.464  <br><br> 

<font size="3"> We hope this content has been useful. If you want to make a donation please click on the button below. It helps us with this website maintenance. 

[![paypal](https://www.paypalobjects.com/en_US/i/btn/btn_donateCC_LG.gif)](https://www.paypal.com/cgi-bin/webscr?cmd=_s-xclick&hosted_button_id=29CVY97MEQ9BY)